In [14]:
model_dir = "/root/models/bart_eval_3_12_23"

In [5]:
from pynvml import *

def print_gpu_utilization():
    nvmlInit()
    handle = nvmlDeviceGetHandleByIndex(0)
    info = nvmlDeviceGetMemoryInfo(handle)
    print(f"GPU memory occupied: {info.used//1024**2} MB.")


def print_summary(result):
    print(f"Time: {result.metrics['train_runtime']:.2f}")
    print(f"Samples/second: {result.metrics['train_samples_per_second']:.2f}")
    print_gpu_utilization()
    
print_gpu_utilization()

GPU memory occupied: 438 MB.


In [13]:
from transformers import pipeline
import numpy as np
import pandas as pd
from sklearn import metrics
import torch
import datasets
from datasets import Dataset
from transformers.pipelines.pt_utils import KeyDataset
from tqdm.auto import tqdm
import os
os.environ['DISABLE_MLFLOW_INTEGRATION'] = 'TRUE'

print_gpu_utilization()

GPU memory occupied: 438 MB.


In [7]:
from transformers import AutoTokenizer
from transformers import AutoModelForSequenceClassification

tokenizer = AutoTokenizer.from_pretrained('facebook/bart-large-mnli')
model = AutoModelForSequenceClassification.from_pretrained("facebook/bart-large-mnli")
print_gpu_utilization()
model.config.label2id

GPU memory occupied: 438 MB.


{'contradiction': 0, 'entailment': 2, 'neutral': 1}

In [28]:
train_tokenized = datasets.load_from_disk("/root/data/bart_train_labels")
val_tokenized = datasets.load_from_disk("/root/data/bart_val_labels")

from datasets import concatenate_datasets
train_concat = concatenate_datasets([train_tokenized['train'], train_tokenized['val']])
val_concat = concatenate_datasets([val_tokenized['train'], val_tokenized['val']])


small_train_dataset = train_concat.shuffle(seed=42)
small_val_dataset = val_concat.shuffle(seed=42).select(range(10))

In [29]:
small_val_dataset

Dataset({
    features: ['id', 'labels', 'original_label', 'input_ids', 'attention_mask'],
    num_rows: 10
})

In [30]:
small_val_dataset[0]

{'id': 63252,
 'labels': 1,
 'original_label': 'Pleural Effusion',
 'input_ids': [0,
  46444,
  4139,
  16632,
  19,
  431,
  750,
  9,
  19258,
  29048,
  19961,
  29382,
  4,
  440,
  26606,
  92,
  22628,
  13581,
  4,
  26449,
  27005,
  775,
  58,
  7123,
  8,
  3694,
  24785,
  58,
  156,
  11,
  42,
  507,
  266,
  25,
  3905,
  35,
  20,
  181,
  9475,
  611,
  8307,
  337,
  5963,
  1043,
  2192,
  1153,
  33,
  103,
  1076,
  548,
  19231,
  7681,
  53,
  24,
  34,
  45,
  1714,
  11,
  204,
  377,
  536,
  4,
  2,
  2,
  713,
  1246,
  16,
  18486,
  9799,
  20029,
  15727,
  4,
  2,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1

In [31]:
from transformers import TrainingArguments, Trainer
import numpy as np
import evaluate

metric = evaluate.load("accuracy")

def compute_metrics(eval_pred):
    logits_tuple, labels = eval_pred
    logits, _ = logits_tuple
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)


training_args = TrainingArguments(
    output_dir=f"{model_dir}/output",
    evaluation_strategy="epoch",
    num_train_epochs=1,
    logging_steps=10,
    per_device_train_batch_size=3,
    per_device_eval_batch_size=3,
    gradient_accumulation_steps=10, # effective batch size is per_device_train_batch_size * gradient_accumulation_steps
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=small_train_dataset,
    eval_dataset=small_val_dataset,
    compute_metrics=compute_metrics,
)

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


In [32]:
predictions = trainer.predict(small_val_dataset)

The following columns in the test set don't have a corresponding argument in `BartForSequenceClassification.forward` and have been ignored: original_label, id. If original_label, id are not expected by `BartForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Prediction *****
  Num examples = 10
  Batch size = 3


In [33]:
predictions

PredictionOutput(predictions=(array([[-0.28392026,  1.3950751 , -0.72586584],
       [ 0.55211437,  1.4536133 , -1.7330351 ],
       [ 0.21891618,  2.1311917 , -1.8585143 ],
       [ 1.5499318 , -0.08018561, -1.3137777 ],
       [ 1.6641283 ,  0.46099567, -2.1147885 ],
       [-0.74595684,  3.1285396 , -1.6388547 ],
       [-0.23591323,  2.0054336 , -1.5462494 ],
       [ 0.9626813 ,  1.3985592 , -2.0485644 ],
       [ 1.1258554 ,  0.8481924 , -1.4403977 ],
       [-0.24244455,  2.167667  , -1.6237764 ]], dtype=float32), array([[[ 4.2885300e-03,  4.3158934e-02,  3.3609826e-02, ...,
          1.7105922e-02,  7.3895748e-03, -1.5797011e-03],
        [-2.3670703e-01, -1.6069575e-01, -2.0428053e-01, ...,
         -1.4110659e-01,  6.1788846e-02,  2.2416133e-02],
        [ 1.1391005e-01, -4.0776384e-01,  9.4448857e-02, ...,
          1.0185369e-01,  5.8663897e-02, -1.9737989e-01],
        ...,
        [ 4.2885300e-03,  4.3158945e-02,  3.3609815e-02, ...,
          1.7105933e-02,  7.3895520e-0